### Data Extraction From IMDB an Search it in Kids in mind 

In [2]:
import requests
from bs4 import BeautifulSoup

#### Step 2
upload the source code file if there are restriction on web page 
other wise the coe for web scraping


In [3]:
with open('movies.txt','r') as f:
    html_content=f.read()
print(html_content)




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
        
<script type='text/javascript'>var ue_t0=ue_t0||+new Date();</script>
<script type='text/javascript'>
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);


    var ue_err_chan = 'jserr';
(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b.logLevel;c&&c!==k&&c!==m&&c!==n&&c!==p||a.ec++;c&&c

#### step 3
Parse the html (break info in to strings)

In [4]:
soup=BeautifulSoup(html_content,'html.parser')

#### step 4
fetch the data 

In [5]:
# fetch the titile of the html
title=soup.title
print("Ttile: ",title.text)

# now fetch the info we need 
movie_info=[]

#all the information we need to fetch is in the tbody 
tbody=soup.find("tbody",{"class":"lister-list"})
tbody
#now as there are many tr tags for each movie we apply for loop
rows = tbody.find_all('tr')
for row in rows:
    # there 3 attributes in titleColumn are movie name ,rank, year
    column=row.find('td',{"class":"titleColumn"})
   
    # rank is tricky, as it is 1,2 or 3 digits before so we fetch yp to 4 digits 
    rank=column.text.strip()[0:4]
    # Now from the first 4, fetch the rank starting from index 0 to the index of . 
    rank=rank[0:rank.index('.')]

    # name is TEXT in the A tag of Column.
    movieName=column.a.text
    
    # to fetch year 
    # Year is in Column's SPAN tag and its surrounded wiht () so used strip to remove that.
    year = column.span.text.strip('()')
    
    # Rating is in another column (td) as text and we need to use strip to remove extra spaces/
    rating = row.find('td', class_="ratingColumn imdbRating").text.strip()
    
    movie_info.append([rank,movieName,year,rating])
    
    

Ttile:  Top 250 Movies - IMDb


In [6]:
# now import pandas, from pandas we can see our data in to tabular form
import pandas as pd
df=pd.DataFrame(movie_info,columns=['Rank','Movie_Name','Year_release','IMDB_Ratings'])
df
# create new columns for data we fetch from KIM
df["SexNudity"]=''
df['Violence']=''
df['Profanity']=''
df['KIM Rating']=''

# save to CSV file
df.to_csv('IMDB_Movies.csv')


In [7]:
df

,Rank,Movie_Name,Year_release,IMDB_Ratings,SexNudity,Violence,Profanity,KIM Rating
0,1,The Shawshank Redemption,1994,9.2,,,,
1,2,The Godfather,1972,9.2,,,,
2,3,The Dark Knight,2008,9.0,,,,
3,4,The Godfather Part II,1974,9.0,,,,
4,5,12 Angry Men,1957,9.0,,,,
...,...,...,...,...,...,...,...,...
245,246,The Help,2011,8.0,,,,
246,247,The Iron Giant,1999,8.0,,,,
247,248,Life of Brian,1979,8.0,,,,
248,249,Aladdin,1992,8.0,,,,


In [8]:
def KIMmovies(txt):
    import re
    infoList = {}
    # first we are finding the year, that we will need to find the rating
    # Group was used, as the regex object returned 2/3 groups, so our required data was in group index 1. 
    # in order to understand, just print the varaible "m" and we will see.
    m = re.search(r"\[([0-9]+)\]", txt)
    if not m:
        year = 'N/a'
    else:
        year = m.group(1)    
 
    # pick benchmark score for each category SexNudity / Violence / Profanity 
    # this is FINDING 3 numbers with . in between and + means it can be single digit 
    # like (1.2.1) or more digit like (10.10.9) "\d" <-- this means digit 
    # some movies has no block of these benchmark that is why we also need to check if that block exists 
    # that is why I used IF below.
    # Finally we are telling that fetch the first group of this regex, there can be more on the same page 
    # group() is used because of following
    # bm is <re.Match object; span=(27, 32), match='1.7.4'>
    # bm.group() is the required bm pattern that we need.
    bm = re.search(r'\d+.\d+.\d+', txt)
    if not bm:
        bm = 'N/a'
    else:
        #print(bm.group())
        bm = bm.group()
        
    # here we are removing the year found above and benchmark scores from the string so that 
    # we will only left with movie name and rating
    movie = txt.replace('['+year+']','').replace('-','').replace(bm,'')
    # now check and pring the original variable (txt) and this movie, you will see that no it only has 
    # name of the movie and rating
    # this below regex we use to grab the rating, this can be a number, or alphabets OR alpha-num like PG13, of PG-13
    # there for we are trying to find just ANYTHING withint the square brackets. 
    # then again, checking if it has any match, then grab ii and it is in the group 1, that is why we pick group(1)
    # IF you want to know why did i use group, just print the rating var before I write .group(1)
    # rating is <re.Match object; span=(11, 17), match='[PG13]'>
    # rating.group() is [PG13]
    # rating.group(1) is [PG13]
 
    
    rating = re.search(r"\[(.*?)\]", movie)
    if not rating:
        rating = 'N/a'
    else:
        #print(rating.group(1))
        rating = rating.group(1)
    movie = movie.replace('['+rating+']','')
    
    # here we are finding the rating 
    movie = txt.replace('['+year+']','').replace('-','').replace(bm,'')
    rating = re.search(r"\[(.*?)\]", movie)
    if not rating:
        rating = 'N/a'
    else:
        rating = rating.group(1)        
        
    # now that we have the bm, we check if this is not NA, thne break it in to 3 categories 
    if not bm == 'N/a':
        bm = bm.split('.')
        try:
            infoList['SexNudity'] = bm[0]
        except:
            infoList['SexNudity'] = ''

        try:
            infoList['Violence'] = bm[1]
        except:
            infoList['Violence'] = ''

        try:
            infoList['Profanity'] = bm[2] 
        except:
            infoList['Profanity'] = ''                   
    else:
        infoList['SexNudity'] = ''
        infoList['Violence'] = ''
        infoList['Profanity'] = ''    
    infoList['Rating'] = rating

    return infoList

In [11]:
# Now Scrae the Kids in mind Data
def dataKIM(mn,y,df,i):
    import requests
    from bs4 import BeautifulSoup
    # URL for the KIM data
    url = f"https://kids-in-mind.com/search-desktop.htm?fwp_keyword={mn}+{y}"
    html = requests.get(url)
    if html.status_code == 200: 
        soup = BeautifulSoup(html.content,"html.parser")
        div = soup.find("div",{'class':'fwpl-item el-2onu0j search-result'})
        data = div.find('a').text
        info = KIMmovies(data)
        print(mn,info)
        #categorize the data in different genre
        df.at[i,'SexNudity'] = info['SexNudity']
        df.at[i,'Violence'] = info['Violence']
        df.at[i,'Profanity'] = info['Profanity']
        df.at[i,'KIM Rating'] = info['Rating']

In [10]:
# appy for loop for all the rows
for index, row in df.iterrows():
        dataKIM(row.Movie_Name,row.Year_release,df,index)

The Shawshank Redemption {'SexNudity': '7', 'Violence': '7', 'Profanity': '7', 'Rating': 'R'}
The Godfather {'SexNudity': '1', 'Violence': '7', 'Profanity': '1', 'Rating': 'PG13'}
The Dark Knight {'SexNudity': '2', 'Violence': '7', 'Profanity': '3', 'Rating': 'PG13'}
The Godfather Part II {'SexNudity': '1', 'Violence': '7', 'Profanity': '1', 'Rating': 'PG13'}
12 Angry Men {'SexNudity': '8', 'Violence': '8', 'Profanity': '5', 'Rating': 'R'}
Schindler's List {'SexNudity': '6', 'Violence': '10', 'Profanity': '5', 'Rating': 'R'}
The Lord of the Rings: The Return of the King {'SexNudity': '1', 'Violence': '7', 'Profanity': '1', 'Rating': 'PG13'}
Pulp Fiction {'SexNudity': '9', 'Violence': '10', 'Profanity': '10', 'Rating': 'R'}
The Lord of the Rings: The Fellowship of the Ring {'SexNudity': '1', 'Violence': '7', 'Profanity': '1', 'Rating': 'PG13'}
The Good, the Bad and the Ugly {'SexNudity': '1', 'Violence': '6', 'Profanity': '1', 'Rating': 'PG'}
Forrest Gump {'SexNudity': '4', 'Violence': 

In [12]:
df

,Rank,Movie_Name,Year_release,IMDB_Ratings,SexNudity,Violence,Profanity,KIM Rating,SexNuity
0,1,The Shawshank Redemption,1994,9.2,,7,7,R,7
1,2,The Godfather,1972,9.2,,7,1,PG13,1
2,3,The Dark Knight,2008,9.0,,7,3,PG13,2
3,4,The Godfather Part II,1974,9.0,,7,1,PG13,1
4,5,12 Angry Men,1957,9.0,,8,5,R,8
...,...,...,...,...,...,...,...,...,...
245,246,The Help,2011,8.0,,9,6,R,9
246,247,The Iron Giant,1999,8.0,,5,3,PG,2
247,248,Life of Brian,1979,8.0,,5,5,R,6
248,249,Aladdin,1992,8.0,,3,0,N/a,1


In [58]:
df.to_csv('IMDB_Movies.csv')

#### This Section is for my own working process

In [13]:
import streamlit as st

In [24]:
def scrapeKIM(df):
    url =f"https://kids-in-mind.com/search-desktop.htm?fwp_keyword= {df.Movie_Name}+{df.Year_release}"
    print(url)

In [25]:
df.apply(scrapeKIM, axis=1)

https://kids-in-mind.com/search-desktop.htm?fwp_keyword= The Shawshank Redemption+1994
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= The Godfather+1972
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= The Dark Knight+2008
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= The Godfather Part II+1974
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= 12 Angry Men+1957
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= Schindler's List+1993
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= The Lord of the Rings: The Return of the King+2003
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= Pulp Fiction+1994
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= The Lord of the Rings: The Fellowship of the Ring+2001
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= The Good, the Bad and the Ugly+1966
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= Forrest Gump+1994
https://kids-in-mind.com/search-desktop.htm?fwp_keyword= S

0      None
1      None
2      None
3      None
4      None
       ... 
245    None
246    None
247    None
248    None
249    None
Length: 250, dtype: object

In [8]:
def scrapeKIM(Movie_Name):
    url =f"https://kids-in-mind.com/search-desktop.htm?fwp_keyword={Movie_Name}"
    print(url)


In [9]:
scrapeKIM("Inception")


https://kids-in-mind.com/search-desktop.htm?fwp_keyword=Inception


In [10]:
# to check the response whether we can fetch the data or not 
url="https://kids-in-mind.com/search-desktop.htm?fwp_keyword={movie_Name_here}+{year}"
html=requests.get(url)
q=html.status_code
q
#200 means you can fetch it it is 403 then its error 

200

In [11]:
if q==200:
    soup=BeautifulSoup(html.content,'html.parser')
    print(soup)
else:
    print("403 forbidden")

<!DOCTYPE html>

<!--[if IE 6]>
<html id="ie6" lang="en-US" prefix="og: https://ogp.me/ns#">
<![endif]-->
<!--[if IE 7]>
<html id="ie7" lang="en-US" prefix="og: https://ogp.me/ns#">
<![endif]-->
<!--[if IE 8]>
<html id="ie8" lang="en-US" prefix="og: https://ogp.me/ns#">
<![endif]-->
<!--[if !(IE 6) | !(IE 7) | !(IE 8)  ]><!-->
<html lang="en-US" prefix="og: https://ogp.me/ns#">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://kids-in-mind.com/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
	<script src="https://kids-in-mind.com/wp-content/themes/Divi/js/html5.js" type="text/javascript"></script>
	<![endif]-->
<script type="text/javascript">
		document.documentElement.className = 'js';
	</script>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/><style id="et-divi-open-sans-inline-css">/* Original: https://fonts.googleapis.com/css?family=Open+Sans:300italic,400italic,600italic,700italic,800ita

In [12]:
div=soup.find("div",{"class":"fwpl-item el-2onu0j search-result"}).find('a').text
div


'Maps to the Stars [2015] [R] - 10.7.9'

In [ ]:
def evalChunks(txt):
    import re
    infoList = {}
    m = re.search(r"\[([0-9]+)\]", txt)
    if not m:
        year = 'N/a'
    else:
        year = m.group(1)
    bm = re.search(r'\d+.\d+.\d+', txt)
    if not bm:
        bm = 'N/a'
    else:
        bm = bm.group()

    # movie name
    movie = txt.replace('['+year+']','').replace('-','').replace(bm,'')
    rating = re.search(r"\[(.*?)\]", movie)
    if not rating:
        rating = 'N/a'
    else:
        rating = rating.group(1)
    movie = movie.replace('['+rating+']','')
    if not bm == 'N/a':
        bm = bm.split('.')
        infoList['Vulgarity'] = bm[0]
        infoList['Violence'] = bm[1]
        infoList['Profanity'] = bm[2]        
    else:
        infoList['Vulgarity'] = ''
        infoList['Violence'] = ''
        infoList['Profanity'] = ''    
    infoList['Movie'] = movie.strip(' ')
    infoList['Year'] = year
    infoList['Rating'] = rating

    return infoList
evalChunks('Maps to the Stars [2015] [R] - 10.7.9')

In [14]:
import re
movie_name=("Maps to the Stars [2015] [R] - 10.7.9")
result=r'(.*?)\s+[(\d{4})\]\s+[(\d{2})]\s+[(\d{2})]\s+[(\d{2})]
result

SyntaxError: EOL while scanning string literal (3107504186.py, line 3)

In [ ]:
import re
movie="Maps to the Stars [2015] [R] - 10.7.9"
result=re.findall(r'^(.*?)\s*\[(\d{4})\]\s*\[.*?\]\s*-\s*(\d+)\.(\d+)\.(\d+)$',movie)
result

In [ ]:
import re
movie="Maps to the Stars [2015] [R] - 10.7.9"
result=re.findall(r'(\d{4})',movie)`Q r'^(.*?)\s*\ 1
result

In [ ]:
td = soup.find_all("td",{"class":"titleColumn"})

for cell in td:
    MovieName=cell.find('a').text
    print(MovieName)

In [ ]:
def scrapeKIM(df):
    url =f"https://kids-in-mind.com/search-desktop.htm?fwp_keyword={movie_Name_here}+{year}"
    print(url)

In [ ]:
df.apply(scrapeKIM,axis=1)